Treinando o modelo

In [ ]:
# Baixe as dependências
!pip install torch transformers datasets


In [19]:
import os
import json
import torch
from datasets import Dataset, load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments


In [20]:
os.environ["WANDB_DISABLED"] = "true"

In [21]:
# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Treinando na: {device}")

Treinando na: cuda


In [22]:
# Carregar o modelo e tokenizer do Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Mover o modelo para a GPU
model.to(device)

In [24]:
# Carregar seu dataset (substitua com seu caminho correto)
dataset = load_dataset('json', data_files='./text_generator_dataset.json')

In [40]:
# Função de tokenização
def tokenize_function(examples):
    # Tokeniza as perguntas (input) e as respostas (labels)
    question_encodings = tokenizer(examples['question'], padding='max_length', truncation=True, max_length=512)
    answer_encodings = tokenizer(examples['answer'], padding='max_length', truncation=True, max_length=512)

    # Adiciona as labels
    question_encodings['labels'] = answer_encodings['input_ids']

    return question_encodings


In [ ]:
# Aplicar a tokenização
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [42]:
# Dividir o dataset em treino e validação
train_dataset, eval_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1).values()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",              # Diretório de saída
    num_train_epochs=3,                  # Número de épocas
    per_device_train_batch_size=8,       # Tamanho do lote
    per_device_eval_batch_size=8,        # Tamanho do lote para validação
    warmup_steps=500,                    # Passos de aquecimento
    weight_decay=0.01,                   # Decaimento de peso
    logging_dir="./logs",                # Diretório para logs
    fp16=True# Passos para log
)

In [57]:
trainer = Trainer(
    model=model,                         # Seu modelo treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Dataset de treinamento
    eval_dataset=eval_dataset            # Dataset de validação
)

In [ ]:
# Iniciar o treinamento
trainer.train()

# Salvando o modelo no diretório especificado
trainer.save_model()

# Caso queira salvar o tokenizador junto com o modelo:
model.save_pretrained(training_args.output_dir, safe_serialization=True)
tokenizer.save_pretrained(training_args.output_dir)

Utilizando o modelo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Carregar o modelo e o tokenizador
model_name = "./results"  # Caminho do seu modelo treinado
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Função para gerar a resposta
def generate_response(question, answer):
    # Criando o prompt para gerar a frase no formato desejado
    prompt = f"Pergunta: {question}\nResposta: {answer}\nFrase formal: O indicador ROE da empresa ROMI3 é {answer}"

    # Tokenizar o prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Gerar a resposta
    with torch.no_grad():
        generated_ids = model.generate(
            inputs['input_ids'],
            max_new_tokens=50,  # Definir o número máximo de novos tokens que o modelo pode gerar
            num_beams=5,  # Beam search para melhorar a qualidade
            no_repeat_ngram_size=2,  # Evitar repetições
            top_k=50,  # Amostragem
            top_p=0.95,  # Probabilidade acumulada
            temperature=0.7,  # Controle de criatividade
            do_sample=True,  # Ativa amostragem
            early_stopping=True
        )

    # Decodificar a resposta gerada
    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return output

# Exemplo de pergunta e resposta
question = "Qual o valor do indicador ROE da empresa ROMI3?"
answer = "10.25"

# Gerar a resposta
generated_answer = generate_response(question, answer)

# Mostrar a resposta gerada
print(f"Resposta gerada: {generated_answer}")
